In [ ]:
import time
import datetime
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from threading import Thread, Event

class IBKRClient(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.nextValidOrderId = None
        self.realtime_price = None
        self.volume = None
        self.order_ready = Event()

    def nextValidId(self, orderId):
        self.nextValidOrderId = orderId
        self.order_ready.set()

    def error(self, reqId, errorCode, errorString):
        print(f"Error: {reqId}, Code: {errorCode}, Msg: {errorString}")

    def realtimeBar(self, reqId, time, open_, high, low, close, volume, wap, count):
        self.realtime_price = close
        self.volume = volume
        print(f"Real-time price: {close}, Volume: {volume} at {datetime.datetime.fromtimestamp(time)}")

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. Id: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, AvgFillPrice: {avgFillPrice}")
        if status == "Filled":
            print(f"Order filled: {filled} shares at an average price of {avgFillPrice}")

    def place_order(self, contract, action, quantity, order_type="MKT"):
        self.order_ready.wait()
        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = order_type
        self.placeOrder(self.nextValidOrderId, contract, order)
        self.nextValidOrderId += 1
        self.order_ready.clear()

    def market_order(self, contract, action, quantity):
        print(f"Placing market order: {action} {quantity} shares")
        self.place_order(contract, action, quantity, "MKT")

    def twap_order(self, contract, action, quantity, duration_seconds=300):
        print(f"Placing TWAP order: {action} {quantity} shares over {duration_seconds} seconds")
        slices = duration_seconds // 5  # Assuming execution every 5 seconds
        slice_quantity = quantity // slices
        for i in range(slices):
            self.place_order(contract, action, slice_quantity, "MKT")
            time.sleep(5)

    def vwap_order(self, contract, action, quantity, duration_seconds=300):
        print(f"Placing VWAP order: {action} {quantity} shares over {duration_seconds} seconds")
        slices = duration_seconds // 5  # Assuming execution every 5 seconds
        for i in range(slices):
            # Adjust order size based on market volume to simulate VWAP
            slice_quantity = int((self.volume / sum(self.volume for _ in range(slices))) * quantity)
            self.place_order(contract, action, slice_quantity, "MKT")
            time.sleep(5)

    def market_impact_order(self, contract, action, quantity, market_impact_factor=0.1):
        print(f"Placing order with market impact consideration: {action} {quantity} shares")
        adjusted_quantity = max(1, int(quantity * (1 - market_impact_factor * abs(quantity))))
        self.place_order(contract, action, adjusted_quantity, "MKT")

def create_contract(symbol, sec_type, exchange, currency):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.exchange = exchange
    contract.currency = currency
    return contract

def main():
    # Initialize IBKR client
    client = IBKRClient()
    client.connect("127.0.0.1", 4002, clientId=1)  # Port 4002 for paper trading

    # Run the client in a separate thread
    client_thread = Thread(target=client.run)
    client_thread.start()

    # Wait until next valid order ID is received
    client.order_ready.wait()

    # Create contract
    contract = create_contract("AAPL", "STK", "SMART", "USD")

    # Request real-time market data
    client.reqRealTimeBars(1, contract, 5, "TRADES", True, [])

    # User selects the execution algorithm
    execution_algorithm = input("Choose execution algorithm (market, twap, vwap, impact): ").lower()
    action = input("Choose action (BUY or SELL): ").upper()
    quantity = int(input("Enter quantity: "))

    # Execute order based on chosen algorithm
    if execution_algorithm == "market":
        client.market_order(contract, action, quantity)
    elif execution_algorithm == "twap":
        duration_seconds = int(input("Enter duration in seconds for TWAP: "))
        client.twap_order(contract, action, quantity, duration_seconds)
    elif execution_algorithm == "vwap":
        duration_seconds = int(input("Enter duration in seconds for VWAP: "))
        client.vwap_order(contract, action, quantity, duration_seconds)
    elif execution_algorithm == "impact":
        market_impact_factor = float(input("Enter market impact factor (e.g., 0.1): "))
        client.market_impact_order(contract, action, quantity, market_impact_factor)
    else:
        print("Invalid algorithm selected.")

    try:
        while True:
            Event().wait(5)  # Keep the client running
    except KeyboardInterrupt:
        print("Stopping the trading client...")
    finally:
        client.disconnect()
        client_thread.join()

if __name__ == "__main__":
    main()
